<!-- Title Section -->
<div style="display: flex; align-items: center; justify-content: center; border-bottom: 4px solid black; padding-bottom: 10px; margin-bottom: 20px; height: 100px;">
  <h1 style="font-family: 'Times New Roman', serif; font-size: 32px; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; margin-right: 10px;">Sydney University Artificial Intelligence Association</h1>
  <img src="https://usu.edu.au/_gatsby/file/7111e770ffabd38cfe1252cefb30461d/linkedin-1.png?u=https%3A%2F%2Fusu.wpengine.com%2Fwp-content%2Fuploads%2F2025%2F02%2Flinkedin-1.png"/>
</div>

<h2>Workshop 1 - Cascade Classifier with OpenCV</h2>

<!-- OpenCV Section -->
<div style="max-width: 800px; margin-bottom: 40px;">
  <h3>What is OpenCV?</h3>

  <!-- OpenCV logo + Wrapped Text -->
  <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRTu1OmEFc5Dx8-_lQv555XmUisLUI1S0DK-w&s"
  width="200"
  style="float: left; margin-right: 20px; border-right: 2px solid black; padding-right: 20px;"/>

  <p style="text-align: justify;">
  <b>Open Source Computer Vision Library</b>, or commonly known as <b>OpenCV</b>, is a computer vision and machine learning open-source library.
  It is an expansive library capable of complex video processing and object detection capabilities.
  </p>
  <p style="text-align: justify;">
  To get the <a href="https://opencv.org/get-started/">OpenCV</a> software library click on the link and download according to your system. Ask the tutor for assistance if you run into trouble!
  </p>
  <p style="text-align: justify;">
  If you wish to know more about OpenCV, read the <a href="https://opencv.org/about/">official website</a> or a <a href="https://www.geeksforgeeks.org/opencv-overview/">geeksforgeeks</a> summary
  <!-- Clear the float to avoid layout breaking -->
  <div style="clear: both;"></div>
</div>

<!-- Haar Cascade Section -->
<div style="max-width: 800px; margin-bottom: 40px;">
  <h4>Haar Cascade Object Detection</h4>

  <!-- Haar Cascade Text -->
  <p style="text-align: justify;">
  Haar feature-based cascade classifiers was first introduced by <a href="https://ieeexplore.ieee.org/document/990517">Viola and Jones</a> in their 2001 publication <i>'Rapid object detection using a boosted cascade of simple features'</i>, and is an effective object detection algorithm notable for its fast detection capabilities. In a nutshell, the algorithm slides a window which extracts Haar features as shown below. These simplify features by taking the difference of the pixel sum of light and dark regions. Once the features are grouped, the algorithm will apply stages to distinguish regions of interest from background regions.
  </p>
  <!-- Haar Cascade Image -->
  <div style="text-align: center; margin-top: 20px; margin-bottom: 20px;">
    <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQmTo0e9DGxDCXowWRS_0KPcsfgT80yBE8klQ&s" width="500"/>
  </div>
  <p style="text-align: justify;">
  If you are interested in a detailed explanation go to <a href="https://docs.opencv.org/4.x/db/d28/tutorial_cascade_classifier.html">OpenCV</a> documentation or read <a href="https://www.geeksforgeeks.org/face-detection-using-cascade-classifier-using-opencv-python/">GeeksforGeeks</a>' comprehensive guide.
  </p>
</div>

## Exercise: Real-time face detection with python
<p>Now that we know what library and algorithm we will use to detect faces, lets get hands-on with its implementation using Python!</p>
<p>First, make sure to import the necessary libraries:</p>

In [1]:
import os
import cv2

print(cv2.__version__)

4.11.0


<p>Once you have sucessfully imported the libraries, make sure you have the pre-trained <a href="https://github.com/opencv/opencv/tree/master/data/haarcascades">Haar cascades</a>.
</p>
<p>The pre-trained models should come with the installation of OpenCV, so the approach below should work if that is the case. If there are problems finding them, ask your tutor for help.</p>
<p>I decided to use a dictionary to store the model files for each feature and obtain the full path using <code>cv2.data.haarcascades</code>, which provides the base directory.</p>

In [2]:
# Get cv2 default haarcascade directory
data_dir = cv2.data.haarcascades

# Define paths to different haarcascade models
cascade_names = {
    "face": "haarcascade_frontalface_default.xml", 
    "eyes": "haarcascade_eye.xml"
    # "mouth": "haarcascade_mcs_mouth.xml", 
    # "nose": "haarcascade_mcs_nose.xml"
}
cascade_path = {roi: os.path.join(data_dir, filename) for roi, filename in cascade_names.items()}

print(cascade_path)


{'face': '/home/fernando/.local/lib/python3.12/site-packages/cv2/data/haarcascade_frontalface_default.xml', 'eyes': '/home/fernando/.local/lib/python3.12/site-packages/cv2/data/haarcascade_eye.xml'}


<p>Loading the pre-trained Haar cascades is straight-forward, for each feature create a <code>CascadeClassifier()</code> object and load it using the <code>load()</code> method.</p>

In [3]:
#-- 1. Load the cascades
cascades = {}
for roi, path in cascade_path.items():
    if not os.path.exists(path): # Check if the file exists
        print(f"--(!)Warning: {roi} cascade not found at {path}, Skipping...")
        continue # Skip cascade if file is missing

    classifier = cv2.CascadeClassifier()
    if not classifier.load(path):
        print(f'--(X)Error loading {roi} cascade at {path}')
        exit(0)
    cascades[roi] = classifier
print("--(*)Haar cascades loaded successfully!")

--(*)Haar cascades loaded successfully!


<p>The loaded Haar cascade can then make predictions using <code>detectMultiScale()</code> method, which returns boundary rectangles for the ROI.</p>
<p>The function <code>detect()</code> will process the frame by returning bounding boxes for the faces which then can be used to apply the Haar cascades for the eyes and other facial features.
</p>

In [ ]:
def detect(frame):
    # Convert the frame into grayscale and equalize
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)

    #-- Detect faces
    faces = cascades['face'].detectMultiScale(
        frame_gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
        )
    # Loop over the bounding box for each face
    for (x,y,w,h) in faces:
        # Obtain the center and draw an ellipse around the face
        center = (x + w//2, y + h//2)
        frame = cv2.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
        face_roi = frame_gray[y:y+h, x:x+w]
        #-- In each face, detect eyes
        eyes = cascades['eyes'].detectMultiScale(
            face_roi,
            scaleFactor=1.1,
            minNeighbors=10,
            minSize=(15, 15),
            flags=cv2.CASCADE_SCALE_IMAGE
            )
        # Loop over the bounding box for the eyes relative to the original frame dimensions 
        for (x2, y2, w2, h2) in eyes:
            ptA = (x + x2, y + y2)
            ptB = (x + x2 + w2, y + y2 + h2)
            # Draw rectangles around the eyes
            cv2.rectangle(frame, ptA, ptB, (0, 0, 255), 2)
    cv2.imshow('Capture - Face detection', frame) # Show frame

<p>Finally, we can visualize our output using <b>OpenCV's</b> video stream as follows:</p>

In [ ]:
#-- 2. Read the video stream
cap = cv2.VideoCapture(0)
if not cap.isOpened:
    print("--(X)Error opening video capture")
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print("--(!) No captured frame -- Break!")
        break
    detect(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows() # Clean-up

--(!) No captured frame -- Break!


: 